<a href="https://colab.research.google.com/github/JensBlack/MultiAgentBehavior-Challenge/blob/main/MABe_Task_1_Classical_Classification_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![AIcrowd-Logo](https://images.aicrowd.com/raw_images/challenges/banner_file/755/73708a8ac63b521cb3be.jpg)



<p align="center"> Join the communty! <br><a href="https://discord.gg/GTckBMx"><img src="https://img.shields.io/discord/657211973435392011?style=for-the-badge" alt="chat on Discord"></a>
</p>

<h1> <center>
🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐁🐁🐁🐁🐁🐁🐁🐁🐁🐁<br>
🐀 MABe Classical Classification: Starter kit 🐁<br>
🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐁🐁🐁🐁🐁🐁🐁🐁🐁🐁
</center>
</h1>

<p align="center"><img src="https://images.aicrowd.com/uploads/ckeditor/pictures/324/content_task1_structure.png">
</p>

# Setup AIcrowd Utilities 🛠

In [77]:
!pip install -U aicrowd-cli==0.1 > /dev/null 

# Install packages 🗃

Please add all pacakages installations in this section

In [78]:
!pip install numpy pandas

# Import necessary modules and packages 📚


In [79]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

# Download the dataset 📲

Please get your API key from https://www.aicrowd.com/participants/me


In [80]:
API_KEY = "0a318adc55b35d126bcabfb535e0dfda"
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [81]:
!aicrowd dataset download --challenge mabe-task-1-classical-classification

train.npy: 100% 118M/118M [00:02<00:00, 49.8MB/s]
sample-submission.npy: 100% 65.4M/65.4M [00:01<00:00, 45.0MB/s]
test-release.npy: 100% 1.83G/1.83G [00:59<00:00, 31.0MB/s]


Extract the downloaded dataset to `data` directory


In [82]:
!rm -rf data
!mkdir data
 
!mv train.npy data/train.npy
!mv test-release.npy data/test.npy
!mv sample-submission.npy data/sample_submission.npy

# Load Data
The dataset files are python dictionaries, [this](https://colab.research.google.com/drive/1ddCX-TAdEcsUaGf09f5Glgr_G57FMK_O#scrollTo=JPsfxdl2GMcM&line=18&uniqifier=1) is a descirption of how the data is organized.



In [83]:
train = np.load('data/train.npy',allow_pickle=True).item()
test = np.load('data/test.npy',allow_pickle=True).item()
sample_submission = np.load('data/sample_submission.npy',allow_pickle=True).item()

## Dataset Specifications 💾

-   **`train.npy`** - Training set for the task, which follows the following schema:

<p align="left" style="padding: 30px">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img width="600px" src="https://i.imgur.com/GddvoT2.png">
</p>


-   **`test-release.npy`** - Test set for the task, which follows the following schema :

<p align="left" style="padding: 30px">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img width="600px" src="https://i.imgur.com/vfa8Y4V.png">
</p>

-   **`sample_submission.npy`** - Template for a sample submission which follows the following schema

```json
{
    "<sequence_id-1>" : [0, 0, 1, 2, ...],
    "<sequence_id-2>" : [0, 1, 2, 0, ...]
}
```

Each key in the dictionary here refers to the unique sequence id obtained for the sequences in the test set.
The value for each of the keys is expected to hold a list of corresponing annotations. The annotations are represented by the index of the corresponding annotation words in the vocabular provided in the test set.

## How does the data look like? 🔍

### Data overview

In [84]:
print("Dataset keys - ", train.keys())
print("Vocabulary - ", train['vocabulary'])
print("Number of train Sequences - ", len(train['sequences']))
print("Number of test Sequences - ", len(test['sequences']))

Dataset keys -  dict_keys(['vocabulary', 'sequences'])
Vocabulary -  {'attack': 0, 'investigation': 1, 'mount': 2, 'other': 3}
Number of train Sequences -  70
Number of test Sequences -  458


### Sample overview

In [85]:
sequence_names = list(train["sequences"].keys())
sequence_key = sequence_names[0]
single_sequence = train["sequences"][sequence_key]
print("Sequence name - ", sequence_key)
print("Single Sequence keys ", single_sequence.keys())
print(f"Number of Frames in {sequence_key} - ", len(single_sequence['annotations']))
print(f"Keypoints data shape of {sequence_key} - ", single_sequence['keypoints'].shape)
print(f"annotator_id of {sequence_key} - ", single_sequence['annotator_id'])

Sequence name -  1cac195d39
Single Sequence keys  dict_keys(['keypoints', 'annotator_id', 'annotations'])
Number of Frames in 1cac195d39 -  4879
Keypoints data shape of 1cac195d39 -  (4879, 2, 2, 7)
annotator_id of 1cac195d39 -  0


# Convert to pandas


In [86]:
sequence_names = list(train["sequences"].keys())
sequence_key = sequence_names[0]
single_sequence = train["sequences"][sequence_key]
single_keypoints = single_sequence['keypoints']
single_annotations = single_sequence['annotations']
print(np.unique(single_annotations))
single_keypoints_2d = single_keypoints.reshape(single_keypoints.shape[0], single_keypoints.shape[1]*single_keypoints.shape[2]*single_keypoints.shape[3])
print(single_keypoints.shape, single_keypoints_2d.shape)

[1 2 3]
(4879, 2, 2, 7) (4879, 28)


In [87]:
#convert to pd.DataFrame format
keypoint_names = ['nose', 'ear_left', 'ear_right', 'neck', 'hip_left', 'hip_right', 'tail_base']
keypoints_idx = pd.MultiIndex.from_product([['animal1', 'animal2'],keypoint_names, list('xy')], names=['animal','keypoints','coords'])
df_single_keypoints = pd.DataFrame(single_keypoints_2d, columns = keypoints_idx)
df_single_keypoints[('classification', '', 'idx')] = single_annotations
df_single_keypoints.head()

animal        animal1              ...     animal2 classification
keypoints        nose              ...   tail_base               
coords              x           y  ...           y            idx
0          298.993678  266.993678  ...  487.872987              3
1          295.167458  272.167458  ...  488.832877              3
2          301.360918  278.360918  ...  488.252561              3
3          301.279237  283.279237  ...  487.633701              3
4          314.116591  292.116591  ...  485.935361              3

[5 rows x 29 columns]

# Preprocess

We'll normalize the data based on the information that the frame size is 1024x570


In [88]:
#modified from baseline model to fit to pandas
def normalize_data(df):
  df_norm = df.copy()
  df_norm.loc[:, df_norm.columns.get_level_values(2)=='x'] = df_norm.loc[:, df_norm.columns.get_level_values(2)=='x']/1024
  df_norm.loc[:,df_norm.columns.get_level_values(2)=='y'] = df_norm.loc[:, df_norm.columns.get_level_values(2)=='y']/570
  return df_norm

In [89]:
norm_df = normalize_data(df_single_keypoints)
norm_df.head()

animal      animal1                      ...   animal2           classification
keypoints      nose            ear_left  ... tail_base                         
coords            x         y         x  ...         x         y            idx
0          0.291986  0.468410  0.243158  ...  0.423704  0.855918              3
1          0.288249  0.477487  0.252117  ...  0.423665  0.857602              3
2          0.294298  0.488352  0.258165  ...  0.424075  0.856583              3
3          0.294218  0.496981  0.262968  ...  0.423470  0.855498              3
4          0.306754  0.512485  0.270622  ...  0.420835  0.852518              3

[5 rows x 29 columns]

# Go through all sequences

In [90]:
## Going through all sequences and creating giant dataframe
keypoint_names = ['nose', 'ear_left', 'ear_right', 'neck', 'hip_left', 'hip_right', 'tail_base']
keypoints_idx = pd.MultiIndex.from_product([['animal1', 'animal2'],keypoint_names, list('xy')], names=['animal','keypoints','coords'])

collection_df = None
for sequence_idx in range(len(sequence_names)):
  sequence_key = sequence_names[sequence_idx]
  single_sequence = train["sequences"][sequence_key]
  single_keypoints = single_sequence['keypoints']
  single_annotations = single_sequence['annotations']
  single_keypoints_2d = single_keypoints.reshape(single_keypoints.shape[0], single_keypoints.shape[1]*single_keypoints.shape[2]*single_keypoints.shape[3])

  # convert to dataframe
  df_single_keypoints = pd.DataFrame(single_keypoints_2d, columns = keypoints_idx)
  df_single_keypoints[('classification', '', 'idx')] = single_annotations
  df_single_keypoints[('classification', '', 'idx')] = df_single_keypoints[('classification', '', 'idx')].astype('category')

  #normalize coordinates
  df_single_keypoints = normalize_data(df_single_keypoints)

  if collection_df is None:
    collection_df = df_single_keypoints
  else:
    collection_df = pd.concat([collection_df, df_single_keypoints], ignore_index= True)

#collection_df[('classification', '', 'idx')] = collection_df[('classification', '', 'idx')].astype("category")
collection_df.describe(include= 'all')


animal           animal1                 ...        animal2 classification
keypoints           nose                 ...      tail_base               
coords                 x              y  ...              y            idx
count      507738.000000  507738.000000  ...  507738.000000  507738.000000
mean            0.487286       0.863737  ...       0.586393       2.283101
std             0.269802       0.475637  ...       0.235602       0.972861
min             0.000000       0.019298  ...       0.029825       0.000000
25%             0.222908       0.370868  ...       0.365867       1.000000
50%             0.491596       0.888416  ...       0.608195       3.000000
75%             0.754293       1.344355  ...       0.811078       3.000000
max             0.994514       1.739340  ...       0.998992       3.000000

[8 rows x 29 columns]

In [91]:
# take equally sized samples from all categories:
#feature_df = collection_df.loc[collection_df[('classification', '', 'idx')] != 3]
feature_df = collection_df.copy()
label_count= feature_df[('classification', '', 'idx')].value_counts()
min_label = label_count.idxmin() 
min_count = np.amin(label_count)
feature__sample = feature_df.groupby(('classification', '', 'idx')).sample(n=min_count, random_state=14)
feature__sample.shape

print(f'Minimum occurances of label {min_label}: {min_count}. Taking {min_count} samples per group...')

Minimum occurances of label 0: 14043. Taking 14043 samples per group...


# Basic prediction: Single frame - single prediction with Random Forest (no additional features)

In [92]:
from sklearn.model_selection import train_test_split
labels = np.array(feature__sample[('classification', '', 'idx')])
# Remove the labels from the features
features = feature__sample.drop(('classification', '', 'idx'), axis = 1)
# Saving feature names
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 14)
print('Training Features :', train_features.shape)
print('Training Labels :', train_labels.shape)
print('Testing Features :', test_features.shape)
print('Testing Labels :', test_labels.shape)

Training Features : (42129, 28)
Training Labels : (42129,)
Testing Features : (14043, 28)
Testing Labels : (14043,)


In [93]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators = 200, random_state = 14)

# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)

In [94]:
predictions = rf.predict(test_features)

# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(test_labels, predictions))

0.9074271879228085


# Generate predictions 💪

Note to myself:
DON'T FORGET TO ALTER THE TEST FILE THE SAME YOU DID THE TRAIN FILE!!!


In [95]:
# Generating Random Predictions
submission = {}
test = np.load('data/test.npy',allow_pickle=True).item()
for sequence_id, sequence in test["sequences"].items():
  single_keypoints = sequence['keypoints']
  single_keypoints_2d = single_keypoints.reshape(single_keypoints.shape[0], single_keypoints.shape[1]*single_keypoints.shape[2]*single_keypoints.shape[3])
  # convert to dataframe
  df_single_keypoints = pd.DataFrame(single_keypoints_2d, columns = keypoints_idx)
  #normalize coordinates
  df_single_keypoints = normalize_data(df_single_keypoints)
  test_features = np.array(df_single_keypoints)




  submission[sequence_id] = rf.predict(test_features)
  

## Validate the submission ✅
The submssion should follow these constraints:

1.   It should be a dictionary
2.   It should be have same keys as sample_submission
3.   The lengths of the arrays are same
4.   All values are intergers

You can use the helper function below to check these



In [96]:
def validate_submission(submission, sample_submission):
    if not isinstance(submission, dict):
      print("Submission should be dict")
      return False

    if not submission.keys() == sample_submission.keys():
      print("Submission keys don't match")
      return False
    
    for key in submission:
      sv = submission[key]
      ssv = sample_submission[key]
      if not len(sv) == len(ssv):
        print(f"Submission lengths of {key} doesn't match")
        return False
    
    for key, sv in submission.items():
      if not all(isinstance(x, (np.int32, np.int64, int)) for x in list(sv)):
        print(f"Submission of {key} is not all integers")
        return False
    
    print("All tests passed")
    return True

In [97]:
validate_submission(submission, sample_submission)

All tests passed


True

## Save the prediction as `npy` 📨

In [98]:
np.save("submission.npy", submission)

## Submit to AIcrowd 🚀

In [99]:
!aicrowd submission create -c mabe-task-1-classical-classification -f submission.npy

submission.npy ━━━━━━━━━━━━━━━━━━━━━━ 100.0% • 65.4/65.4 MB • 2.3 MB/s • 0:00:00
                                                                    ╭─────────────────────────╮                                                                    
                                                                    │ Successfully submitted! │                                                                    
                                                                    ╰─────────────────────────╯                                                                    
                                                                          Important links                                                                          
┌──────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/multi-agent-behavior-representation-modelin